In [1]:
import os,sys
sys.path.append('misc/lib/python3.7/site-packages')
#PigmentHunter is added for a file path

from IPython.display import display
import math
import numpy as np
import nglview as nv
import matplotlib.pyplot as plt
import parmed as pmd
import numpy.random as random
import re
import requests
from hubzero.submit.SubmitCommand import SubmitCommand
import time
import hublib.tool as tool
import hublib.ui as ui
import ipywidgets as widgets


from IPython.display import Javascript, display


class ExcStruc:
    def __init__(self):
        self.nres = 0
        self.resnums = []
        self.restypes = []
        self.ham = []
        self.dips = []
        self.cents = []
            
    def reset(self):
        self.nres = 0
        self.resnums = []
        self.restypes = []
        self.ham = []
        self.dips = []
        self.cents = []

class RepList:
    def __init__(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def reset(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def append(self, nrep):
        self.names.append(nrep.name)
        ptext = {"type": nrep.type, "params": {"color": nrep.color, "sele": nrep.selection, "opacity": str(nrep.opacity)}}
        self.params.append(ptext)
        self.reps.append(nrep)
        
class NGLRep:
    def __init__(self, name, rtype, sel, col, opac):
        self.name = name
        self.type = rtype
        self.selection = sel
        self.color = col
        self.opacity = opac

def check_atoms(RefAtNames, QuerAtNames):
    ismatch = True
    for atnm in RefAtNames:
        if(QuerAtNames.count(atnm)!=1):
            ismatch = False
    return ismatch

def find_porph(pmdstruc, xtruc):
    PORatList = list(['NA', 'NB', 'NC', 'ND'])
    for n in range(0, len(pmdstruc.residues)):
        r = pmdstruc.residues[n]
        nmlist = list()
        for at in r:
            nmlist.append(at.name)
        if(check_atoms(PORatList, nmlist)):
            if(nmlist.count('MG')==1):
                xstruc.resnums.append(n)
                xstruc.nres += 1 
                xstruc.restypes.append('CHL')
                
            else:
                xstruc.resnums.append(n)
                xstruc.nres += 1 
                xstruc.restypes.append('PHO')

    return


def find_dipoles(struc,xtruc):
    ResNums = xtruc.resnums
    DipMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        for at in r:
            if at.name=='NB':
                NB = struc.coordinates[at.idx]
            if at.name=='ND':
                ND = struc.coordinates[at.idx]
        DipMat[n,:] = NB - ND
    xtruc.dips = DipMat

def find_centers(struc, xtruc):
    ResNums = xtruc.resnums
    CentMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        cent = 0.0
        for at in r:
            if at.name=='NB':
                cent += 0.5*struc.coordinates[at.idx]
            if at.name=='ND':
                cent += 0.5*struc.coordinates[at.idx]
        CentMat[n,:] = cent
    xtruc.cents = CentMat

def sync_widgets_to_rep(rep):
    seldrop.value = rep.name
    styledrop.value = rep.type
    colordrop.value = rep.color
    opacslide.value = int(rep.opacity*100)
    
def sync_rep_to_widgets():
    global rList
    num = rList.names.index(seldrop.value)
    rep = rList.reps[num]
    rep.type = styledrop.value
    rep.name = seldrop.value
    rep.color = colordrop.value
    rep.opacity = opacslide.value*0.01
    ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
    rList.params[num] = ptext
    pdbview.set_representations(rList.params)
    

def std_rep(nglview, nvstruc, xstruc):
    global rList
    pdbview.add_trajectory(struc)
    pdbview.clear(0)
    chltxt = ''
    photxt = ''
    for n in range(0, len(xstruc.resnums)):
        if xstruc.restypes[n]=='CHL':
            if len(chltxt)>0:
                chltxt += ' or '
            chltxt += str(xstruc.resnums[n]+1)
        if xstruc.restypes[n]=='PHO':
            if len(photxt)>0:
                photxt += ' or '
            photxt += str(xstruc.resnums[n]+1)
    
    rList.append(NGLRep("Protein", "cartoon", "protein", "grey", 0.2))
    if len(chltxt)>0:
        rList.append(NGLRep("Chlorophyll", "licorice", chltxt, "green", 1.0))
    if len(photxt)>0:
        rList.append(NGLRep("Pheophytin", "licorice", photxt, "blue", 1.0))
    
    pdbview.set_representations(rList.params)
    
    seldrop.options=rList.names
    seldrop.disabled=False
    
    styledrop.disabled=False
    
    colordrop.disabled=False
    opacslide.disabled=False
    sync_widgets_to_rep(rList.reps[0])
    
    for n in range(0, np.shape(xstruc.dips)[0]):
        cent = xstruc.cents[n,:]
        dip = xstruc.dips[n,:]
        v1 = cent - 1.5*dip
        v2 = cent + 1.5*dip
        pdbview.shape.add_arrow(v1.tolist(), v2.tolist(), [1,0,0 ], 1.0 )
        
        
pdbid = widgets.Text(
    value='2DRE',
    placeholder='Type something',
    description='PDB ID:',
    layout = widgets.Layout(width='4cm'),
    disabled=False
)

pdbgo = widgets.Button(
    description='Display',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

pdboutput = widgets.HTML(
    value="",
    placeholder='',
    description='',
)

Output = widgets.Output()

def clear_stage(view):
    view._clear_component_auto_completion()
    if view._trajlist:
        for traj in view._trajlist:
            view._trajlist.remove(traj)
    for component_id in view._ngl_component_ids:
        component_index = view._ngl_component_ids.index(component_id)
        view._ngl_component_ids.remove(component_id)
        view._ngl_component_names.pop(component_index)
        view._remote_call('removeComponent',
            target='Stage',
            args=[component_index,])
    # Shapes appear not to be populated in component_id list.
    # So we have to call seperately removeAllComponents to 
    # get rid of them. 
    view._remote_call('removeAllComponents',
            target='Stage',
            args=[0])
    view._update_component_auto_completion()


def pdbgo_onclick(b):
    global struc
    global estruc
    url = 'http://files.rcsb.org/download/'+pdbid.value+'.pdb'
    r = requests.get(url, allow_redirects=True)
    if(r.status_code==200):
        fname = pdbid.value + '.pdb'
        wfd = open(fname, 'wb')
        wfd.write(r.content)
        wfd.close()
        pdboutput.value = ''
        
        xstruc.reset()
        clear_stage(pdbview)
        rList.reset()
        struc = pmd.load_file(fname)
        find_porph(struc, xstruc)
        find_dipoles(struc, xstruc)
        find_centers(struc, xstruc)
        std_rep(pdbview, struc, xstruc)

    else:
        pdboutput.value = 'Please enter a valide PDB ID code.'

        
xstruc = ExcStruc()
pdbgo.on_click(pdbgo_onclick)
pdbid.on_submit(pdbgo_onclick)
pdbview = nv.NGLWidget()
pdbview._set_size('500px', '500px')
pdbbox = widgets.HBox([pdbid, pdbgo])
rList = RepList()


seldrop = widgets.Dropdown(
    options=rList.names,
    #value='Protein',
    description='Selection:',
    disabled=True,
)


styledrop = widgets.Dropdown(
    options=['cartoon', 'licorice', 'spacefill'],
    description='Style:',
    disabled=True,
)

colordrop = widgets.Dropdown(
    options=['chain', 'red', 'green', 'blue', 'grey'],
    description='Color:',
    disabled=True,
)

opacslide = widgets.IntSlider(
    min=0,
    max=100,
    step=1,
    description='Opacity:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

dispbox = widgets.VBox([seldrop, styledrop, colordrop, opacslide])
viewbox = widgets.HBox([pdbview, dispbox])


def seldrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        rep = rList.reps[rList.names.index(seldrop.value)]
        sync_widgets_to_rep(rep)
        
def styledrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.type = styledrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)
        
def colordrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.color = colordrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)
        
def opacslide_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.opacity = opacslide.value*0.01
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)

seldrop.observe(seldrop_on_change)
styledrop.observe(styledrop_on_change)
colordrop.observe(colordrop_on_change)
opacslide.observe(opacslide_on_change)
    
inprogress = False
        
display(pdbbox)
display(pdboutput)
display(viewbox)
struc = pmd.structure

<IPython.core.display.Javascript object>

HTML(value='', placeholder='')

In [78]:
class amide(object): #defines amide class
    def __init__(self,C,N,H,O):
        self.C=C
        self.N=N
        self.H=H
        self.O=O
   # am=amide()
    


In [484]:
matrix_bond=[]
struc2=struc[":ALA, ARG, ASN, ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL"] #isolates amino acids
struc2=struc2["@N,O,C,H"] #pulls atoms labeled N, C, H, O ignoring all other atoms
for r in struc2.residues:
    for at in r:
        if np.size(at.bond_partners) == 2: #atoms bonded too two other atoms in our remaining atoms will only include C and N if H is included and only C if no H in model
            bond_data=at.bond_partners #pulls bonding information
            
            O_num=str(bond_data[0])#isolates atom numbers from bonding data
            [ignore, O_num]=O_num.split("[")
            [O_num, ignore]=O_num.split("]")
            N_num=str(bond_data[1])
            [ignore, N_num]=N_num.split("[")
            [N_num, ignore]=N_num.split("]")  
            C_num=str(at) #have to do this because at.number alway reports -1
            [ignore, C_num]=C_num.split("[")
            [C_num, ignore]=C_num.split("]")
            matrix_bond.append([int(O_num),int(N_num),int(C_num)])

Amid_matrix=[]
for n in range (0,len(matrix_bond)):
    bond_number=matrix_bond[n]
    if np.size(bond_number)==3:
        Amid_matrix.append(amide(bond_number[2],bond_number[1],"",bond_number[0]))
    if np.size(bond_number)==4:
        Amid_matrix.append(amide(bond_number[2],bond_number[1],bond_number[3],bond_number[0]))



                
          

59.09